# Vergabe NRW CSV file -  07 /01/2021

### What does this script do?
It collects all data available at the API daten.vergabe.nrw.de and saves it into a csv file

I would like to thank Alex Voigt (Github @avqpereira ) and Thiago Alencar (Github @thiagoalencar ) for helping me create this script. This is also result of their work =)

In [18]:
import json
import requests
import math
import pandas as pd
import janitor

In the next block, we will define the number of results that will be collected. This is necessary to stablish following loop:

In [5]:
#setando a url de busca 
url = 'https://daten.vergabe.nrw.de/rest/evergabe/aggregation_search'

#qtde de dados que retornará a cada consulta
size = 5000

#query:
payload = '{"sort":[{"_id":"asc"}],"query":{"match_all":{}},"size":'+str(size)+'}'

#headers necessários à consulta da API
headers = {
    'accept': "application/json",
    'content-type': "application/json"
 #   'cache-control': "no-cache"
    }

#o que cai juntar todos os vergabe no DF final:
tenders_array = []

#requisição:
response = requests.request("POST", url, data=payload, headers=headers)

#dados em formato json
query_data = json.loads(response.text)

tenders_array.extend(query_data['results'])

#vendo total de resultados:
total_hits = query_data['pagination']['total']
result_size = len(query_data['results'])
last_id = query_data['results'][-1]["_id"]

number_of_loops = ((total_hits - size) // size )
last_loop_size = ((total_hits - size) % size) # explicação abaixo:

Cheking total hits:

In [12]:
total_hits

47502

Some explanations about the code to myself:

* The modulus operator, sometimes also called the remainder operator or integer remainder operator works on integers (and integer expressions) and yields the remainder when the first operand is divided by the second. In Python, the modulus operator is a percent sign ( % ).
* json.loads() method can be used to parse a valid JSON string and convert it into a Python Dictionary
* extend() is a built-in Python function that adds the specified list elements (or any iterable) to the end of the current list. 

Above, the loop to gather all data available:

In [9]:
#O loop com todos:

for i in range(number_of_loops+1):
    if i == number_of_loops:
        size=last_loop_size
    payload = '{"sort":[{"_id":"asc"}],"query":{"match_all":{}},"size":'+str(size)+',"search_after":["'+last_id+'"]}'
    response = requests.request("POST", url, data=payload, headers=headers)
    query_data = json.loads(response.text)
    result_size = len(query_data['results'])
    if result_size > 0:
        tenders_array.extend(query_data['results'])
        last_id = query_data['results'][-1]["_id"] 
    else:
        break

Checking if all the collected data into array matches the total_hits

In [11]:
len(tenders_array)

#funcinou!

47502

Nor I'll transform it into dataframes:

In [19]:
#Transformando em Dataframe:

df = pd.json_normalize(tenders_array)

#limpando nomes:
nomes_originais = df.columns
nomes = df.columns
df.columns = nomes
df = df.clean_names()

Checking:

In [20]:
df[1:10]

,_id,created_at,updated_at,result_id,result_title,result_description,result_procedure_type,result_order_type,result_publication_date,result_cpv_codes,...,result_buyer_postal_code,result_seller_name,result_seller_town,result_seller_country,result_geo_lon,result_geo_lat,result_value,result_created_at,result_updated_at,result_buyer_country
1,CXPNY42D474,2021-05-26T22:20:10.194Z,2021-06-08T22:19:38.411Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-26T22:20:10.194Z,2021-06-08T22:19:38.411Z,NaN
2,CXPNY42D4QL,2021-05-26T22:18:23.840Z,2021-05-31T22:17:09.010Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-26T22:18:23.840Z,2021-05-31T22:17:09.010Z,NaN
3,CXPNY42DR40,2020-05-11T00:12:51.144Z,2020-06-26T23:21:14.223Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-11T00:12:51.144Z,2020-06-26T23:21:14.223Z,NaN
4,CXPNY42DRDN,2020-04-20T22:14:01.166Z,2020-05-19T00:33:44.632Z,CXPNY42DRDN,Videobeobachtung Dortmund Münsterstraße,"[Software, Hardware, Installations- und Montag...",Öffentliche Ausschreibung,UVGO,,[75131000-3],...,44139,,,,7.46023,51.49958,,2020-04-20T22:14:01.166Z,2020-05-19T00:33:44.632Z,NaN
5,CXPNY42DYUZ,2020-01-25T02:18:23.855Z,2020-01-30T02:18:08.235Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-25T02:18:23.855Z,2020-01-30T02:18:08.235Z,NaN
6,CXPNY42YT80,2019-12-18T23:01:46.907Z,2020-01-04T23:01:27.747Z,CXPNY42YT80,Polizei Dortmund sucht Pächter für Kantine,[Bewirtschaftung der auf der Liegenschaft Mark...,Öffentliche Ausschreibung,OTHER,,"[55512000-2, 55511000-5, 55510000-8]",...,44139,,,,7.46023,51.49958,,2019-12-18T23:01:46.907Z,2020-01-04T23:01:27.747Z,NaN
7,CXPNY42YWAX,2019-12-19T01:46:19.436Z,2019-12-19T01:46:19.436Z,CXPNY42YWAX,NaN,[],,,,[45216111],...,44139,,,,7.46023,51.49958,,2019-12-19T01:46:19.436Z,2019-12-19T01:46:19.436Z,DE
8,CXPNY43D5NQ,2020-11-24T23:02:58.968Z,2021-02-22T23:00:23.152Z,CXPNY43D5NQ,Beschaffung von VMware vSphere 7 Lizenzen und ...,[8 x VMware vSphere 7 Enterprise Plus 1 Proces...,Ex post Veröffentlichung (§ 30 Abs. 1),UVGO,,[48000000-8],...,44608,,,,8.05830,48.21670,,2020-11-24T23:02:58.968Z,2021-02-22T23:00:23.152Z,NaN
9,CXPNY43D626,2020-08-17T22:01:51.564Z,2020-10-06T22:01:42.803Z,CXPNY43D626,Unterhalts- und Glasreinigungsleistungen,[Die zu beauftragenden Reinigungsleistungen um...,Ex post Veröffentlichung (§ 30 Abs. 1),UVGO,,"[90911300-9, 90911200-8]",...,44608,,,,8.05830,48.21670,,2020-08-17T22:01:51.564Z,2020-10-06T22:01:42.803Z,NaN


We have here some blank and NaN data. What does that mean? Is there any difference between blanked and NaN data?

Saving df:

In [21]:
#Exportando em CSV
df.to_csv(r'df_vergabe_nrw_jan2021.csv')